## Create "Model Topper" using example

In [37]:

import os

import numpy as np
import pandas as pd

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense,GlobalAveragePooling2D
from keras import applications

In [38]:
# dimensions of our images.
img_width, img_height = 299, 299

# dimensions of our images.
img_width, img_height = 299, 299
n_features = 6

top_model_weights_path = 'bottleneck_xception_model.h5'
train_data_dir = 'multi_class_testing/train/'
validation_data_dir = 'multi_class_testing/test/'

available_train_files = len(os.listdir(train_data_dir + 'female/')) \
    + len(os.listdir(train_data_dir + 'male/'))
available_test_files = len(os.listdir(validation_data_dir + 'female/')) \
    + len(os.listdir(validation_data_dir+'male/'))
    
nb_train_samples  = available_train_files - available_train_files % 8
nb_validation_samples = available_test_files - available_test_files % 8

epochs = 2
batch_size = 8

__Set up Multiclass Testing__

In [39]:
watch_df = pd.read_csv('all_watch_info_with_indicators.csv')
watch_dict = dict()

Replace These Features with something better

In [40]:
for image_name in watch_df['image_name']:
    watch_dict[image_name+'.jpg'] =  watch_df[
        watch_df['image_name'] == image_name][
            ['is_gold','is_silver','is_strap','is_square','is_round','is_male']].values[0]
    

Multiclass Helper Functions

In [41]:
def multiclasses_getter(x, i, gen):
    count = 0
    y = np.zeros((batch_size, n_features), dtype=np.int8)
    
    # Feature not working
    if np.shape(x)[0] < gen.batch_size:
        steps = np.shape(x)[0]
    else:
        steps = gen.batch_size
        
    idx = (i) * gen.batch_size
    
    for f in gen.filenames[idx : idx + steps]:
        k = f.split('/')[-1]
        y[count,:] = watch_dict[k] 
        count += 1
    return y

In [42]:
def multiclass_flow_train(flow_from_directory_gen, multiclasses_getter):
    for i, (x, y) in enumerate(flow_from_directory_gen):
        yield x, multiclasses_getter(x, i,train_generator)
    
def multiclass_flow_test(flow_from_directory_gen, multiclasses_getter):
    for i, (x, y) in enumerate(flow_from_directory_gen):
        yield x, multiclasses_getter(x, i, test_generator)
      

In [43]:
datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    shuffle=False)

test_generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    shuffle=False)

Found 7336 images belonging to 2 classes.
Found 3144 images belonging to 2 classes.


Load Standard Xception Model

In [44]:
model = applications.xception.Xception(include_top=False, weights='imagenet')

Build My "Custom" Model

In [45]:
watch_model = Sequential()
watch_model.add(model)
watch_model.add(GlobalAveragePooling2D(name='avg_pool'))
watch_model.add(Dense(6, activation="sigmoid"))
watch_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, None, None, 2048)  20861480  
_________________________________________________________________
avg_pool (GlobalAveragePooli (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 12294     
Total params: 20,873,774
Trainable params: 20,819,246
Non-trainable params: 54,528
_________________________________________________________________


Freeze Exception Layers

In [46]:
for layer in watch_model.layers[0].layers:
    layer.trainable = False

Compile

In [47]:
watch_model.compile(loss = "binary_crossentropy", optimizer='sgd', metrics=["accuracy"])

Fit 

In [48]:
# fine-tune the model
watch_model.fit_generator(
        multiclass_flow_train(train_generator, multiclasses_getter),
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data = multiclass_flow_test(test_generator, multiclasses_getter),
        validation_steps=nb_validation_samples // batch_size)

Epoch 1/2
917/917 [==============================] - 452s - loss: 0.3797 - acc: 0.8196 - val_loss: 0.5285 - val_acc: 0.7938
Epoch 2/2
917/917 [==============================] - 449s - loss: 0.0843 - acc: 0.9774 - val_loss: 0.0200 - val_acc: 1.0000


In [49]:
watch_model.save('xception_multiclass1.h5')